In [33]:
# Byte pair encoding algorithm for tokenisation was popularised by GPT


# Tokenisation can be a problem for the following things
#     Spelling words
#     Reversing words and other sipmle string processing
#     Worse in non english languages such as Chinese and Japanese
#     More trouble coding in specific languages such as python
#         For example programming in python we waste context length because spaces in python for indentation are all separate tokens
#     Trailing whitespace problems where other tokens are generated

# Increasing number of tokens is not necessarily always better otherwise the embedding table and the softmax will always grow
# Should be appropriately dense e.g not having spaces for every single indentation in python - in new tokenisers they are represented in a single token
# These choices are deliberate when designing the tokenisers as it densifies python and it can attend to more code before it - the coding ability does not just rely on the architecture but also the tokeniser design 

"""
Hello how are you hope you are well
15496, 703, 389, 345, 2911, 345, 389, 880
"""


'\nHello how are you hope you are well\n15496, 703, 389, 345, 2911, 345, 389, 880\n'

In [34]:
# Bit pair algorithm implementation using UTF

# Strings must be tokenised into a given vocabulary that can be fed into a transformer as input
# We want to support all languages and other special characters such as emojis
    
# The vocabulary in unicode is not a stable representation and it already continues changing it is also very very long
# UTF-8 is the standard into an 8 byte string 2^8

# We can use the byte pairing algorithm that will allow us to use variable length - compressed training dataset
tokens = list("this is simply a test of unicode encoding in python, the longer the string the better we can test our bit pair algorithm - notice the amount of 'the' used. GPT Generated Sentece for testing - The quick brown fox jumps over the lazy dog! こんにちは世界! Добро пожаловать! ¡Hola, mundo! 你好，世界! Bonjour le monde! 안녕하세요 세계! Γειά σου Κόσμε! שלום עולם! नमस्ते दुनिया! 🌍🚀💻✨ नमस्ते, मेरा नाम है GPT-4. This is a long sentence designed to test a tokenizer's capabilities. 一只敏捷的棕色狐狸跳过了懒狗! हरिओम, यहां हम विभिन्न भाषाओं और लिपियों का प्रयोग कर रहे हैं। Lorem ipsum dolor sit amet, consectetur adipiscing elit. カタカナとひらがなも使います。 Моя цель — проверить токенизацию. ¿Puedes entender este texto? 😊✨👾🎉 Python is great for scripting! எங்கள் விஞ்ஞானிகள் நியூயார்க்கில் உள்ளனர். الطقس جميل اليوم! Будем рады видеть вас снова. ここに多くの異なる文字があります。 Это предложение становится длиннее и длиннее. 我们正在测试各种字符。 Δοκιμάζουμε διαφορετικούς χαρακτήρες. הקפיצה המהירה של השועל החום מעל הכלב העצלן! Всем привет! 🌟🌐📚👩‍💻🧑‍🚀🎨 βελτιώνουμε συνεχώς το μοντέλο μας. ¿Qué tal tu día? မင်္ဂလာပါ။ हमने बहुत सारी भाषाएँ शामिल की हैं। ताजमहल भारत में है। 🚗🚀📱💡💬🌈🙌 Этот текст продолжает расти. Qu'est-ce que vous en pensez? 今日はどうですか? Aloha ʻāina! फिर मिलेंगे। 🏖️🏔️🗽🕌🏯 🚴‍♂️🏊‍♀️⛷️🏋️‍♀️🤹‍♂️".encode("utf-8"))

def get_pairs(data):
    counts = {}
    # Here we can use zip to iterate over consecutive elements
    for pair in zip(data, data[1:]):
        # We get the value, by default if it does not exist it will be 0 and we will add 1
        counts[pair] = counts.get(pair, 0) + 1
    return counts

pairs = get_pairs(tokens)
# We can use sorted to sort by the value by using a for loop
# By default python will make it descending so we can use reversed
sorted_pairs = sorted(((v, k) for k, v in pairs.items()), reverse=True)
sorted_pairs 

# ord() and chr() are inverses
# (2, (116, 104)) is the most common pair
# This being t followed by h
# chr(116)
# chr(104)

# Here we can use a dictionary to get the max via a key function that is .get
# we can use the .get function taht will compare all the values in the 
# by giving it the key= function this function pairs.get will return the value that should be filtered
top_pair = max(pairs, key=pairs.get)
top_pair

print(top_pair)
len(tokens)

(224, 164)


2171

In [35]:
def merge(ids, pair, idx):
    # replace all occurrences of pairs with that id with the new index token
    newids = []

    i = 0
    while i < len(ids):
        # The only time we dont want to do it is when we are at the last position
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids 

# tokens = merge(tokens, top_pair, 256)

# We have reduced the amount of tokens used by using repeat occurrences byte pair
# len(tokens)

In [36]:
# This byte pair algorithm has a hyper parameter for tokenisation where we can actually dictate how many steps we want to run it for
#     The longer we run it for the larger the vocabulary but the shorter our encoding is so there are always tradeoffs
#     GPT4 uses 100k tokens

# Now that we have defined the code we can include it in a loop

vocabulary_size = 276
num_merges = vocabulary_size - 256

# We copy the original list here so that we don't destroy it
ids = list(tokens)

# Keep track of our merges
merges = {}
for i in range(num_merges):
    # Can optimise this function later by making it so that it will get the top one and replace all the ones that match based on the same number of pair values instead of just one
    pairs = get_pairs(ids)
    pair = max(pairs, key = pairs.get)
    # We can choose a new token that is 256 values higher than the current one - since we are using utf 8 it will be 256 + any value will always be out of the range
    idx = 256 + i
    ids = merge(ids, pair, idx)
    merges[pair] = idx

# Here idx is the new token that we will be replacing the pair with
# ids is the list of the values that is being replaced
# pair is the max pair that needs to be replaced

# from 2000 to 1700 which is great
# The greater the vocabulary elements the greater the compression ratio
ids

[116,
 104,
 105,
 115,
 32,
 105,
 115,
 32,
 115,
 105,
 109,
 112,
 108,
 121,
 32,
 97,
 268,
 101,
 115,
 116,
 32,
 111,
 102,
 32,
 117,
 110,
 105,
 99,
 111,
 100,
 264,
 270,
 99,
 111,
 100,
 105,
 110,
 103,
 32,
 105,
 110,
 32,
 112,
 121,
 116,
 104,
 111,
 110,
 44,
 268,
 104,
 264,
 108,
 111,
 110,
 103,
 101,
 114,
 268,
 104,
 264,
 115,
 116,
 114,
 105,
 110,
 103,
 268,
 104,
 264,
 98,
 101,
 116,
 116,
 101,
 114,
 32,
 119,
 264,
 99,
 97,
 110,
 268,
 101,
 115,
 116,
 32,
 111,
 117,
 114,
 32,
 98,
 105,
 116,
 32,
 112,
 97,
 105,
 114,
 32,
 97,
 108,
 103,
 111,
 114,
 105,
 116,
 104,
 109,
 32,
 45,
 32,
 110,
 111,
 116,
 105,
 99,
 264,
 116,
 104,
 264,
 97,
 109,
 111,
 117,
 110,
 116,
 32,
 111,
 102,
 32,
 39,
 116,
 104,
 101,
 39,
 32,
 117,
 115,
 101,
 100,
 269,
 71,
 80,
 84,
 32,
 71,
 270,
 101,
 114,
 97,
 116,
 101,
 100,
 32,
 83,
 270,
 116,
 101,
 99,
 264,
 102,
 111,
 114,
 268,
 101,
 115,
 116,
 105,
 110,
 103,
 32,
 45,
 32,


In [40]:
# Tokeniser has its own training set and preprocessing stage, we use byte pair to train the vocabulary
# Once we haev the vocabulary and the merges we can do encoding and decoding
# Tokenisation can take raw text into a token sequence and a token sequence back into raw text

# Not all token sequences are valid utf-8 sequences

# Encoding and Decoding steps
vocabulary = {idx: bytes([idx]) for idx in range(256)}

# For all the pairs of merges
for (p0, p1), idx in merges.items():
    vocabulary[idx] = vocabulary[p0] + vocabulary[p1]

def decode(ids):
    #
    tokens = b"".join(vocabulary[idx] for idx in ids)
    text = tokens.decode("utf-8")
    return text

def encode(text):
    # Get a list of integers from the utf encoding merge based on our trained merges vocabulary
    tokens = list(text.encode("utf-8"))
    # Now we have to implement the byte pair algorithm - since we are doing pairings we have to make sure that the lenght of tokens is at least 2
    while len(tokens) >= 2:
        pairs = get_pairs(tokens)
        # find the key with the lowest index in the array - start with lowest before we work our way onto larger indexes
        # Since we are using .get we can apply a fall back and if we get a pair that is not in the tokenisation language it does not occur and thus by default we set infinity
        # Sort the pairs and return the min given the function that retrieves our token values
        pair = min(pairs, key = lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            # We know that nothing else can be merged if everything is infinity
            break
        idx = merges[pair]
        # Keep merging the pairs using the index token that we have defined in our vocabulary
        tokens = merge(tokens, pair, idx)
    return tokens

In [42]:
# decode(encode("Hello how are you"))

'Hello how are you'

Byte Pair Encoding

The algorithm in GPT-2 is not applied naively, dog will occur frequently next to other punctuation e.g dog. dog? dog! so we will be clustering semantics with punctuation this would be a suboptimal combination
We want to enforce that some times of characters should never be merged together and enforce these merging rules on top of the byte pair algorithm

GPT-2 encoder.py is the tokeniser
OpenAI created a regex pattern that allows them to enforce rules for what parts of the text will never be merged - they used regex package in python


In [44]:
import regex as re 

# Raw string - has things like optional strings 
#  ?\p{L} - a letter from any language one or more times that is following an optional space
#  ?\p{N} - any number so even if we have Helllo345 is split to prevent the tokenisation matching

# re.IGNORECASE would be useful here to make these matches work for upper and lower case versions too HOW'S doesn't work but how's works
# The thing about this pattern is that it is very language specific too so we don't guarantee that this works for other languages
# The last part is matching white space up to the last white space character - "       are", "           you" the extra spaces will be taken except the last one so it will always have a space before words
gpt2_tokenisation_pattern = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2_tokenisation_pattern, "This is a test to find how this regex234 work346s"))
# By splitting spaces and words then this means this regex pattern prevents merges such as across letters, across numbers across punctuation never happen
# ['This', ' is', ' a', ' test', ' to', ' find', ' how', ' this', ' regex', '234', ' work', '346', 's']

['This', ' is', ' a', ' test', ' to', ' find', ' how', ' this', ' regex', '234', ' work', '346', 's']


In [53]:
# import tiktoken

# This is the gpt4 tokeniser
# enc = tiktoken.get_encoding("cl100k_base")
# print(enc.encode("Random tes2345t string!!"))
# tiktoken.list_encoding_names()

# You can download the gpt2 encoder.json which is the vocabulary
# vocab.bpe is the merges that we have made
# With just vocabulary and merges we can encode and decode

[14326, 51309, 11727, 20, 83, 925, 3001]


In [51]:
# Sentence piece can do both train and inference BPE tokenizers 
# used in both llama and mistral series models
# Looks at codepoints are available and starts merging - codepoints will either get mapped to a special unknown token / encode a rare byte_fallback to bytes for rare cases

# When it finds token that it has not seen it it will use byte fallback and if we end up encoding values that are unknown we will get an <unk> token that will delete those values
# spaces end up being defined by _ characters 
# Some tokenisers will end up adding a dummy prefix to the start of sentences so that they are treated the same because "world" and " world" in "hello world" will be treated differently unfortunately                            
tokens = list("this is simply a test of unicode encoding in python, the longer the string the better we can test our bit pair algorithm - notice the amount of 'the' used. GPT Generated Sentece for testing - The quick brown fox jumps over the lazy dog! こんにちは世界! Добро пожаловать! ¡Hola, mundo! 你好，世界! Bonjour le monde! 안녕하세요 세계! Γειά σου Κόσμε! שלום עולם! नमस्ते दुनिया! 🌍🚀💻✨ नमस्ते, मेरा नाम है GPT-4. This is a long sentence designed to test a tokenizer's capabilities. 一只敏捷的棕色狐狸跳过了懒狗! हरिओम, यहां हम विभिन्न भाषाओं और लिपियों का प्रयोग कर रहे हैं। Lorem ipsum dolor sit amet, consectetur adipiscing elit. カタカナとひらがなも使います。 Моя цель — проверить токенизацию. ¿Puedes entender este texto? 😊✨👾🎉 Python is great for scripting! எங்கள் விஞ்ஞானிகள் நியூயார்க்கில் உள்ளனர். الطقس جميل اليوم! Будем рады видеть вас снова. ここに多くの異なる文字があります。 Это предложение становится длиннее и длиннее. 我们正在测试各种字符。 Δοκιμάζουμε διαφορετικούς χαρακτήρες. הקפיצה המהירה של השועל החום מעל הכלב העצלן! Всем привет! 🌟🌐📚👩‍💻🧑‍🚀🎨 βελτιώνουμε συνεχώς το μοντέλο μας. ¿Qué tal tu día? မင်္ဂလာပါ။ हमने बहुत सारी भाषाएँ शामिल की हैं। ताजमहल भारत में है। 🚗🚀📱💡💬🌈🙌 Этот текст продолжает расти. Qu'est-ce que vous en pensez? 今日はどうですか? Aloha ʻāina! फिर मिलेंगे। 🏖️🏔️🗽🕌🏯 🚴‍♂️🏊‍♀️⛷️🏋️‍♀️🤹‍♂️".encode("utf-8"))

['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base', 'o200k_base']

In [ ]:
# Tokenisation

# The more tokens we have the larger our vocabulary size will be and our initial token embedding table will grow as well as our last linear layer that will be predicting the next token in the sentence will be too large
# Too large of chunks the model does not have enough time to process all the links and data as we are compressing too much information at once
# Freezing the base model and training arbitrary parts of it for finde tuning / adding new tokens can be a good idea

# If we have long prompts we can introduce new tokens and train the model by trainig teh new tokens - the behaviour should be identical but we can compress this new prompt to get identical performance called distillation

# Different modality transformers need to have different tokenisations
# We can take an image and truncate it into integers and become tokens
# Visual patches that can be used ans a representation for models with visual data

In [ ]:
# Spelling words can be difficult because characters are broken into tokens that are fairly long
# If we have a single token like .DefaultCellStyle then if we ask the model how many "l" are in it, the model will return the wrong answer

# Other basic mapings such as reversing a string are quite difficult due to tokenisation
# However if you ask it to break up the string with spaces adn then reverse it it can do it perfectly

# Translation can also be quite difficult for models - as we get different token sizes e.g Hello to korean will blow up into 3 tokens 
# Tokenisation can also affect addition as it will have different combinations of characters that are all different sizes e.g 1/324 or 32/52 342/1 separate tokens

# A lot of tokenisation problems with spaces in python or programming is terible since it will dramatically reduce the context length that the model can attend to - a lot is wasted

# We are searching the space of tokens where if retokenised they would be of high probability

# Special tokens such as SolidGoldMagikarp - these tokens are from reddit user names
# The tokenisation dataset was very different from the training dataset - potentially this user may have been a common person that would post a lot this string would occur many times and thus get merged to a single individual token in a vocabulary of 50,000
# This tokenisation dataset has those strings but when you train the model this data was not present and thus it would never be trained as the training and tokenisation data was different
# This token would never be trained - if evoked at run time then it would give undefined behaviour and these weird tokens that have never been trained would be out of distribution